체크포인트를 불러와서 결과를 시각화

In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from pathlib import Path
from tqdm import tqdm
import random

변수 설정

In [2]:
# ===== VISUALIZATION 설정 =====
VISUALIZE_ALL = True  # True: 전체 테스트셋, False: 일부만
NUM_SAMPLES = 20  # VISUALIZE_ALL=False일 때 시각화할 샘플 수
RANDOM_SAMPLE = True  # True: 랜덤 샘플링, False: 순차적

# ===== 실험 설정 =====
DATASET_ROOT = "./dataset"
TEST_DATASET = 'WHU-CD'
TEST_MODEL = 'Changer'  # 모델명
TEST_CLASS = 'ChangerSkeleton'  # 클래스명

경로 설정

In [3]:
# 모델 파일명 변환
model_file = TEST_MODEL.replace('-', '').lower()
model_class = TEST_CLASS

# 클래스명이 모델명과 같으면 모델명만 사용, 다르면 클래스명 사용
if model_class.lower() == TEST_MODEL.lower():
    class_folder = TEST_MODEL
else:
    class_folder = model_class

# 경로 설정: experiments/데이터셋/모델/클래스/
exp_dir = Path(f"experiments/{TEST_DATASET}/{TEST_MODEL}/{class_folder}")
checkpoint_path = exp_dir / "checkpoints" / "best_model.pth"

print(f"Experiment directory: {exp_dir}")
print(f"Checkpoint path: {checkpoint_path}")

Experiment directory: experiments/WHU-CD/Changer/ChangerSkeleton
Checkpoint path: experiments/WHU-CD/Changer/ChangerSkeleton/checkpoints/best_model.pth


GPU

In [4]:
# 단일 GPU 사용
GPU_ID = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using GPU: {GPU_ID}")
print(f"Device: {DEVICE}")

# 멀티 GPU 사용 
# # GPU_IDS = [0, 1, 2, 3]  # 사용할 GPU 리스트
# os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, GPU_IDS))
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# USE_MULTI_GPU = len(GPU_IDS) > 1 and torch.cuda.device_count() > 1
# print(f"Using GPUs: {GPU_IDS}")
# print(f"Available GPU count: {torch.cuda.device_count()}")
# if USE_MULTI_GPU:
#     BATCH_SIZE = BATCH_SIZE * len(GPU_IDS)  # 멀티 GPU시 배치 크기 조정
#     print(f"Adjusted batch size for multi-GPU: {BATCH_SIZE}")
# 시드 설정 (재현가능성)

Using GPU: 1
Device: cuda


In [5]:
# %% 모델 로드
import importlib

def get_model_class(module_name, class_name):
    """모델 동적 import"""
    module_path = f'models.{module_name}'
    
    try:
        module = importlib.import_module(module_path)
        model_class = getattr(module, class_name)
        print(f"✅ Successfully loaded: {class_name} from {module_path}")
        return model_class
    except ImportError as e:
        print(f"❌ Cannot import module {module_path}")
        raise ImportError(f"Module import failed: {e}")
    except AttributeError as e:
        print(f"❌ Class {class_name} not found in {module_path}")
        raise AttributeError(f"Class not found: {e}")

# 모델 로드
ModelClass = get_model_class(model_file, model_class)
model = ModelClass(num_classes=1).to(DEVICE)

checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Model: {ModelClass.__name__}")
print(f"Best F1: {checkpoint.get('best_f1', 0):.4f}")

# %% 데이터로더
from utils import create_dataloaders

_, _, test_loader = create_dataloaders(
    root_dir=DATASET_ROOT,
    dataset_name=TEST_DATASET,
    batch_size=1,
    num_workers=2,
    augment=False
)

total_samples = len(test_loader)
print(f"Test samples: {total_samples}")

✅ Successfully loaded: ChangerSkeleton from models.changer


/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model: ChangerSkeleton
Best F1: 0.7542
Loaded 4233 images from WHU-CD/train
Loaded 747 images from WHU-CD/val
Loaded 2700 images from WHU-CD/test
Test samples: 2700


In [6]:
# # %% 시각화 실행
# from utils.visualization import save_visualization

# print("\n" + "="*50)
# print(f"Starting Visualization")
# print(f"Dataset: {TEST_DATASET}")
# print(f"Model: {TEST_MODEL}")
# print(f"Total samples: {total_samples}")
# print("="*50 + "\n")

# # 샘플 선택
# if VISUALIZE_ALL:
#     sample_indices = list(range(total_samples))
#     print(f"📊 Visualizing ALL {total_samples} samples")
# else:
#     if RANDOM_SAMPLE:
#         sample_indices = sorted(random.sample(range(total_samples), min(NUM_SAMPLES, total_samples)))
#         print(f"📊 Visualizing {len(sample_indices)} RANDOM samples")
#     else:
#         sample_indices = list(range(min(NUM_SAMPLES, total_samples)))
#         print(f"📊 Visualizing first {len(sample_indices)} samples")

# print(f"Sample indices: {sample_indices[:10]}..." if len(sample_indices) > 10 else f"Sample indices: {sample_indices}")
# print()

# # 시각화 실행
# with torch.no_grad():
#     for idx, batch in enumerate(tqdm(test_loader, desc="Visualizing")):
#         # 선택된 샘플만 처리
#         if idx not in sample_indices:
#             continue
        
#         try:
#             # 배치에서 데이터 추출
#             pre_img = batch['img1'].to(DEVICE)
#             post_img = batch['img2'].to(DEVICE)
#             gt_mask = batch['label'].to(DEVICE)
#             filename = batch['filename']
            
#             # 파일명 처리 (배치 크기가 1이므로)
#             if isinstance(filename, (list, tuple)):
#                 file_name = filename[0]
#             else:
#                 file_name = filename
            
#             # 예측
#             pred_mask = model(pre_img, post_img)
            
#             # Sigmoid 적용 (필요한 경우)
#             if pred_mask.max() > 1.0 or pred_mask.min() < 0:
#                 pred_mask = torch.sigmoid(pred_mask)
            
#             # 확장자 제거
#             file_name = Path(file_name).stem
            
#             # 시각화 저장
#             save_visualization(
#                 pre_img=pre_img[0],
#                 post_img=post_img[0],
#                 gt_mask=gt_mask[0],
#                 pred_mask=pred_mask[0],
#                 save_path=str(save_dir),
#                 file_name=file_name
#             )
            
#         except Exception as e:
#             print(f"\n⚠️ Error processing sample {idx}: {e}")
#             import traceback
#             traceback.print_exc()
#             continue

# print("\n" + "="*50)
# print(f"✅ Visualization completed!")
# print(f"📁 Saved to: {save_dir}")
# print(f"📊 Total visualized: {len(sample_indices)} samples")
# print("="*50)

In [ ]:
# %% F1 Score 기반 시각화 (실제 오답이 있는 케이스만)
from utils.visualization import save_visualization
import json

print("\n" + "="*50)
print(f"F1-Score Based Visualization (Samples with Errors)")
print(f"Dataset: {TEST_DATASET}")
print(f"Model: {TEST_MODEL}")
print("="*50 + "\n")

# ===== 설정 =====
EXCLUDE_PERFECT = True  # True: F1=1.0인 샘플 제외
EXCLUDE_NO_CHANGE = True  # True: FP=0, FN=0인 샘플 제외 (변화 없는 정답 케이스)
NUM_WORST_SAMPLES = 400  # 하위 N개
NUM_BEST_SAMPLES = 400   # 상위 N개
# ================


F1-Score Based Visualization (Samples with Errors)
Dataset: WHU-CD
Model: Changer



In [ ]:
# F1 Score 계산 함수
def calculate_f1_score(pred_mask, gt_mask, threshold=0.5):
    """개별 샘플의 F1 Score 계산"""
    pred_binary = (pred_mask > threshold).float()
    gt_binary = (gt_mask > 0.5).float()
    
    # TP, FP, FN, TN 계산
    tp = (pred_binary * gt_binary).sum().item()
    fp = (pred_binary * (1 - gt_binary)).sum().item()
    fn = ((1 - pred_binary) * gt_binary).sum().item()
    tn = ((1 - pred_binary) * (1 - gt_binary)).sum().item()
    
    # F1 계산
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)
    f1 = 2 * precision * recall / (precision + recall + 1e-7)
    
    return f1, precision, recall, tp, fp, fn, tn

# F1 Score 저장용 리스트
sample_scores = []

print("📊 Calculating F1 scores for all samples...")

# 모든 샘플의 F1 Score 계산
with torch.no_grad():
    for idx, batch in enumerate(tqdm(test_loader, desc="Calculating F1")):
        try:
            # 배치에서 데이터 추출
            pre_img = batch['img1'].to(DEVICE)
            post_img = batch['img2'].to(DEVICE)
            gt_mask = batch['label'].to(DEVICE)
            filename = batch['filename']
            
            # 파일명 처리
            if isinstance(filename, (list, tuple)):
                file_name = filename[0]
            else:
                file_name = filename
            
            # 예측
            pred_mask = model(pre_img, post_img)
            
            # Sigmoid 적용
            if pred_mask.max() > 1.0 or pred_mask.min() < 0:
                pred_mask = torch.sigmoid(pred_mask)
            
            # F1 Score 계산
            f1, precision, recall, tp, fp, fn, tn = calculate_f1_score(
                pred_mask[0], gt_mask[0]
            )
            
            # 결과 저장
            sample_scores.append({
                'idx': idx,
                'filename': Path(file_name).stem,
                'f1': f1,
                'precision': precision,
                'recall': recall,
                'tp': tp,
                'fp': fp,
                'fn': fn,
                'tn': tn,
                'batch': batch  # 나중에 시각화할 때 사용
            })
            
        except Exception as e:
            print(f"\n⚠️ Error processing sample {idx}: {e}")
            continue

print(f"\n✅ F1 scores calculated for {len(sample_scores)} samples")

# 필터링
original_count = len(sample_scores)
sample_scores_filtered = sample_scores.copy()

# 1. F1 = 1.0인 완벽한 샘플 제외
if EXCLUDE_PERFECT:
    before = len(sample_scores_filtered)
    sample_scores_filtered = [s for s in sample_scores_filtered if s['f1'] < 0.9999]
    excluded_perfect = before - len(sample_scores_filtered)
    print(f"\n🔍 Perfect samples (F1 ≈ 1.0) excluded: {excluded_perfect}")

# 2. FP=0, FN=0인 샘플 제외 (변화 없는 정답 케이스)
if EXCLUDE_NO_CHANGE:
    before = len(sample_scores_filtered)
    sample_scores_filtered = [s for s in sample_scores_filtered if not (s['fp'] == 0 and s['fn'] == 0)]
    excluded_no_change = before - len(sample_scores_filtered)
    print(f"🔍 No-change correct samples (FP=0, FN=0) excluded: {excluded_no_change}")

print(f"   Remaining samples with errors: {len(sample_scores_filtered)}/{original_count}")

if len(sample_scores_filtered) == 0:
    print("\n⚠️ No samples with errors found! All predictions are perfect.")
    print("="*50)
else:
    # F1 Score 기준으로 정렬 (낮은 순서)
    # F1이 같으면 FN이 높은 순서로 (더 많이 놓친 것을 우선)
    sample_scores_sorted = sorted(sample_scores_filtered, key=lambda x: (x['f1'], -x['fn']))
    
    # 통계 출력
    f1_values = [s['f1'] for s in sample_scores_filtered]
    f1_values_all = [s['f1'] for s in sample_scores]
    
    print(f"\n📈 F1 Score Statistics (samples with errors):")
    print(f"  Count: {len(f1_values)}")
    print(f"  Mean: {np.mean(f1_values):.4f}")
    print(f"  Std: {np.std(f1_values):.4f}")
    print(f"  Min: {np.min(f1_values):.4f}")
    print(f"  Max: {np.max(f1_values):.4f}")
    print(f"  Median: {np.median(f1_values):.4f}")
    
    print(f"\n📈 F1 Score Statistics (all samples):")
    print(f"  Count: {len(f1_values_all)}")
    print(f"  Mean: {np.mean(f1_values_all):.4f}")
    
    # 오류 통계
    total_fp = sum([s['fp'] for s in sample_scores_filtered])
    total_fn = sum([s['fn'] for s in sample_scores_filtered])
    avg_fp = total_fp / len(sample_scores_filtered)
    avg_fn = total_fn / len(sample_scores_filtered)
    
    print(f"\n📊 Error Statistics (filtered samples):")
    print(f"  Total FP (False Positives): {int(total_fp)}")
    print(f"  Total FN (False Negatives): {int(total_fn)}")
    print(f"  Average FP per sample: {avg_fp:.2f}")
    print(f"  Average FN per sample: {avg_fn:.2f}")
    print(f"  Total errors: {int(total_fp + total_fn)}")
    
    # Worst 10 출력
    print(f"\n🔴 Worst 10 samples (highest error rate):")
    for i, sample in enumerate(sample_scores_sorted[:10]):
        total_errors = int(sample['fp'] + sample['fn'])
        print(f"  {i+1}. {sample['filename']}")
        print(f"      F1={sample['f1']:.4f}, Prec={sample['precision']:.4f}, Rec={sample['recall']:.4f}")
        print(f"      FP={int(sample['fp'])}, FN={int(sample['fn'])}, Total Errors={total_errors}")
    
    # Best 10 출력 (에러가 있지만 상대적으로 좋은 케이스)
    print(f"\n🟢 Best 10 samples (from samples with errors):")
    for i, sample in enumerate(sample_scores_sorted[-10:]):
        total_errors = int(sample['fp'] + sample['fn'])
        print(f"  {i+1}. {sample['filename']}")
        print(f"      F1={sample['f1']:.4f}, Prec={sample['precision']:.4f}, Rec={sample['recall']:.4f}")
        print(f"      FP={int(sample['fp'])}, FN={int(sample['fn'])}, Total Errors={total_errors}")
    
    # F1 기반 시각화 저장 디렉토리
    f1_vis_dir = exp_dir / "f1_visualization"
    f1_vis_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📊 Visualizing worst {min(NUM_WORST_SAMPLES, len(sample_scores_sorted))} and best {min(NUM_BEST_SAMPLES, len(sample_scores_sorted))} samples...")
    
    # Worst 샘플 시각화
    worst_dir = f1_vis_dir / "worst"
    worst_dir.mkdir(exist_ok=True)
    
    num_worst = min(NUM_WORST_SAMPLES, len(sample_scores_sorted))
    
    with torch.no_grad():
        for rank, sample_info in enumerate(tqdm(sample_scores_sorted[:num_worst], 
                                                 desc="Visualizing worst")):
            batch = sample_info['batch']
            
            pre_img = batch['img1'].to(DEVICE)
            post_img = batch['img2'].to(DEVICE)
            gt_mask = batch['label'].to(DEVICE)
            
            # 예측
            pred_mask = model(pre_img, post_img)
            if pred_mask.max() > 1.0 or pred_mask.min() < 0:
                pred_mask = torch.sigmoid(pred_mask)
            
            # 파일명에 등수 추가
            file_name = f"{rank+1}.{sample_info['filename']}"
            
            # 시각화 저장
            save_visualization(
                pre_img=pre_img[0],
                post_img=post_img[0],
                gt_mask=gt_mask[0],
                pred_mask=pred_mask[0],
                save_path=str(worst_dir),
                file_name=file_name
            )
    
    # Best 샘플 시각화
    best_dir = f1_vis_dir / "best"
    best_dir.mkdir(exist_ok=True)
    
    num_best = min(NUM_BEST_SAMPLES, len(sample_scores_sorted))
    
    with torch.no_grad():
        for rank, sample_info in enumerate(tqdm(sample_scores_sorted[-num_best:], 
                                                 desc="Visualizing best")):
            batch = sample_info['batch']
            
            pre_img = batch['img1'].to(DEVICE)
            post_img = batch['img2'].to(DEVICE)
            gt_mask = batch['label'].to(DEVICE)
            
            # 예측
            pred_mask = model(pre_img, post_img)
            if pred_mask.max() > 1.0 or pred_mask.min() < 0:
                pred_mask = torch.sigmoid(pred_mask)
            
            # 파일명에 등수 추가
            file_name = f"{rank+1}.{sample_info['filename']}"
            
            # 시각화 저장
            save_visualization(
                pre_img=pre_img[0],
                post_img=post_img[0],
                gt_mask=gt_mask[0],
                pred_mask=pred_mask[0],
                save_path=str(best_dir),
                file_name=file_name
            )
    # Best 샘플 시각화
    best_dir = f1_vis_dir / "best"
    best_dir.mkdir(exist_ok=True)
    
    num_best = min(NUM_BEST_SAMPLES, len(sample_scores_sorted))
    
    with torch.no_grad():
        for rank, sample_info in enumerate(tqdm(sample_scores_sorted[-num_best:], 
                                                 desc="Visualizing best")):
            batch = sample_info['batch']
            
            pre_img = batch['img1'].to(DEVICE)
            post_img = batch['img2'].to(DEVICE)
            gt_mask = batch['label'].to(DEVICE)
            
            # 예측
            pred_mask = model(pre_img, post_img)
            if pred_mask.max() > 1.0 or pred_mask.min() < 0:
                pred_mask = torch.sigmoid(pred_mask)
            
            # 원본 파일명 사용
            file_name = sample_info['filename']
            
            # 시각화 저장
            save_visualization(
                pre_img=pre_img[0],
                post_img=post_img[0],
                gt_mask=gt_mask[0],
                pred_mask=pred_mask[0],
                save_path=str(best_dir),
                file_name=file_name
            )
    
    # F1 Score 분포 히스토그램 저장
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 필터링된 샘플 (에러가 있는 샘플만)
    axes[0].hist(f1_values, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
    axes[0].axvline(np.mean(f1_values), color='r', linestyle='--', linewidth=2, label=f'Mean: {np.mean(f1_values):.4f}')
    axes[0].axvline(np.median(f1_values), color='g', linestyle='--', linewidth=2, label=f'Median: {np.median(f1_values):.4f}')
    axes[0].set_xlabel('F1 Score')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'F1 Score Distribution (Samples with Errors, n={len(f1_values)})')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 전체 샘플
    axes[1].hist(f1_values_all, bins=50, edgecolor='black', alpha=0.7, color='lightcoral')
    axes[1].axvline(np.mean(f1_values_all), color='r', linestyle='--', linewidth=2, label=f'Mean: {np.mean(f1_values_all):.4f}')
    axes[1].axvline(np.median(f1_values_all), color='g', linestyle='--', linewidth=2, label=f'Median: {np.median(f1_values_all):.4f}')
    axes[1].set_xlabel('F1 Score')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'F1 Score Distribution (All Samples, n={len(f1_values_all)})')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle(f'{TEST_MODEL} on {TEST_DATASET}', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(f1_vis_dir / 'f1_distribution.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    # F1 Score 상세 정보를 JSON으로 저장
    score_summary = {
        'model': TEST_MODEL,
        'dataset': TEST_DATASET,
        'total_samples': len(sample_scores),
        'samples_with_errors': len(sample_scores_filtered),
        'perfect_samples': len(sample_scores) - len(sample_scores_filtered) if EXCLUDE_PERFECT else 0,
        'exclude_perfect': EXCLUDE_PERFECT,
        'exclude_no_change': EXCLUDE_NO_CHANGE,
        'error_statistics': {
            'total_fp': int(total_fp),
            'total_fn': int(total_fn),
            'avg_fp_per_sample': float(avg_fp),
            'avg_fn_per_sample': float(avg_fn),
            'total_errors': int(total_fp + total_fn)
        },
        'statistics_filtered': {
            'mean': float(np.mean(f1_values)),
            'std': float(np.std(f1_values)),
            'min': float(np.min(f1_values)),
            'max': float(np.max(f1_values)),
            'median': float(np.median(f1_values)),
            'q1': float(np.percentile(f1_values, 25)),
            'q3': float(np.percentile(f1_values, 75))
        },
        'statistics_all': {
            'mean': float(np.mean(f1_values_all)),
            'std': float(np.std(f1_values_all)),
            'min': float(np.min(f1_values_all)),
            'max': float(np.max(f1_values_all)),
            'median': float(np.median(f1_values_all)),
            'q1': float(np.percentile(f1_values_all, 25)),
            'q3': float(np.percentile(f1_values_all, 75))
        },
        'worst_10': [
            {
                'rank': i+1,
                'filename': s['filename'],
                'f1': float(s['f1']),
                'precision': float(s['precision']),
                'recall': float(s['recall']),
                'tp': int(s['tp']),
                'fp': int(s['fp']),
                'fn': int(s['fn']),
                'tn': int(s['tn']),
                'total_errors': int(s['fp'] + s['fn'])
            }
            for i, s in enumerate(sample_scores_sorted[:10])
        ],
        'best_10': [
            {
                'rank': i+1,
                'filename': s['filename'],
                'f1': float(s['f1']),
                'precision': float(s['precision']),
                'recall': float(s['recall']),
                'tp': int(s['tp']),
                'fp': int(s['fp']),
                'fn': int(s['fn']),
                'tn': int(s['tn']),
                'total_errors': int(s['fp'] + s['fn'])
            }
            for i, s in enumerate(sample_scores_sorted[-10:])
        ]
    }
    
    with open(f1_vis_dir / 'f1_scores_summary.json', 'w', encoding='utf-8') as f:
        json.dump(score_summary, f, indent=2, ensure_ascii=False)
    
    print("\n" + "="*50)
    print(f"✅ F1-based visualization completed!")
    print(f"📁 Saved to: {f1_vis_dir}")
    print(f"  - Worst {num_worst} samples: {worst_dir}")
    print(f"  - Best {num_best} samples: {best_dir}")
    print(f"  - F1 distribution plot: f1_distribution.png")
    print(f"  - Summary JSON: f1_scores_summary.json")
    print(f"\n📊 Filtering Summary:")
    print(f"  - Total samples: {original_count}")
    print(f"  - Samples with errors: {len(sample_scores_filtered)} ({len(sample_scores_filtered)/original_count*100:.1f}%)")
    print(f"  - Perfect samples excluded: {original_count - len(sample_scores_filtered)}")
    print("="*50)

📊 Calculating F1 scores for all samples...


Calculating F1: 100%|██████████| 2700/2700 [00:21<00:00, 126.21it/s]



✅ F1 scores calculated for 2700 samples

🔍 Perfect samples (F1 ≈ 1.0) excluded: 0
🔍 No-change correct samples (FP=0, FN=0) excluded: 1823
   Remaining samples with errors: 877/2700

📈 F1 Score Statistics (samples with errors):
  Count: 877
  Mean: 0.3754
  Std: 0.4129
  Min: 0.0000
  Max: 0.9995
  Median: 0.0000

📈 F1 Score Statistics (all samples):
  Count: 2700
  Mean: 0.1219

📊 Error Statistics (filtered samples):
  Total FP (False Positives): 2113855
  Total FN (False Negatives): 1487347
  Average FP per sample: 2410.32
  Average FN per sample: 1695.95
  Total errors: 3601202

🔴 Worst 10 samples (highest error rate):
  1. test_0009_0001
      F1=0.0000, Prec=0.0000, Rec=0.0000
      FP=0, FN=55943, Total Errors=55943
  2. test_0056_0020
      F1=0.0000, Prec=0.0000, Rec=0.0000
      FP=0, FN=31368, Total Errors=31368
  3. test_0014_0017
      F1=0.0000, Prec=0.0000, Rec=0.0000
      FP=0, FN=25012, Total Errors=25012
  4. test_0014_0018
      F1=0.0000, Prec=0.0000, Rec=0.0000
   

Visualizing worst:  37%|███▋      | 149/400 [02:13<03:37,  1.15it/s]